# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [14]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
def get_file_path_list():
    """
    Create and return list of all files stored in /event_data directory.

    :return: list of full path file names
    """

    # checking your current working directory
    print(os.getcwd())

    # Get your current folder and sub folder event data
    filepath = os.getcwd() + '/event_data'

    # Create a for loop to create a list of files and collect each filepath
    for root, dirs, files in os.walk(filepath):
        # join the file path and roots with the subdirectories using glob
        fp_list = glob.glob(os.path.join(root, '*'))

    return fp_list

In [22]:
file_path_list = get_file_path_list()
#print(file_path_list)

/home/workspace


In [23]:
print(len(file_path_list))

30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [24]:
def get_full_data_rows_list(fp_list):
    """
    Create and return list of data rows read from file list parameter.

    :param fp_list: file path list
    :return: list of data rows
    """

    # initiating an empty list of rows that will be generated from each file
    fd_rows_list = []

    # for every filepath in the file path list
    for f in fp_list:
        # reading csv file
        with open(f, 'r', encoding='utf8', newline='') as csv_file:
            # creating a csv reader object
            csv_reader = csv.reader(csv_file)
            next(csv_reader)

            # extracting each data row one by one and append it
            for line in csv_reader:
                # print(line)
                fd_rows_list.append(line)

    return fd_rows_list

In [25]:
def create_event_data_file_new(fd_rows_list):
    """
    Create new event data file with specific columns from original file
    
    :param fd_rows_list:
    :return:
    """

    # creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
    # Apache Cassandra tables

    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open('event_datafile_new.csv', 'w', encoding='utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
                         'level', 'location', 'sessionId', 'song', 'userId'])
        for row in fd_rows_list:
            if row[0] == '':
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


# Part II. Create new event data file

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [12]:
def read_event_data():
    """
    Iterate through new event data file
    
    :return: current record from event data file
    """
    
    file = 'event_datafile_new.csv'
    with open(file, 'r', encoding='utf8') as f:
        csv_reader = csv.reader(f)
        next(csv_reader)  # skip header
        for r in csv_reader:
            yield r

In [27]:
# get list of fully qualified filenames
file_path_list = get_file_path_list()

# populate data rows from file_path_list into new list
full_data_rows_list = get_full_data_rows_list(file_path_list)

/home/workspace


In [28]:
print(len(full_data_rows_list))

8056


In [29]:
# create new event data file
create_event_data_file_new(full_data_rows_list)

In [30]:
# check the number of rows in your csv file
line_count = 0
for line in read_event_data():    
    line_count+=1

print(line_count)

6820


# Part III. Cassandra database setup and queries

#### Creating a Cluster

In [46]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [47]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

In [48]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

---
### Query 1
#### Return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [49]:
query = "CREATE TABLE IF NOT EXISTS music_library "
query += "(session_id int, item_in_session int, artist varchar, song varchar, length float, \
PRIMARY KEY (session_id, item_in_session))"

print(query)

CREATE TABLE IF NOT EXISTS music_library (session_id int, item_in_session int, artist varchar, song varchar, length float, PRIMARY KEY (session_id, item_in_session))


In [50]:
try:
    session.execute(query)
except Exception as e:
    print(e)

In [51]:
query = "INSERT INTO music_library (session_id, item_in_session, artist, song, length) "
query = query + "VALUES (%s, %s, %s, %s, %s)"

print(query)

INSERT INTO music_library (session_id, item_in_session, artist, song, length) VALUES (%s, %s, %s, %s, %s)


In [52]:
for line in read_event_data():
    try:
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
    except Exception as e:
        print(e)

#### Query 1 Description
<p>In this query, I used session_id as the partition key and item_in_session as my clustering key. Each partition is uniquely identified by session_id. The session_id and item_in_session uniquely identify rows within a partition.  This query returns artist, song, and length from the music_library table for a session_id of 338 and item_in_session of 4.
</p>

In [53]:
# SELECT statement to verify the data was entered into the table
query = "select artist, song, length from music_library where session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


---
### Query 2
#### Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [54]:
query = "CREATE TABLE IF NOT EXISTS music_library_user "
query += "(user_id int, session_id int, item_in_session int, artist varchar, song varchar, first_name varchar, \
last_name varchar, PRIMARY KEY ((user_id, session_id), item_in_session))"

print(query)

CREATE TABLE IF NOT EXISTS music_library_user (user_id int, session_id int, item_in_session int, artist varchar, song varchar, first_name varchar, last_name varchar, PRIMARY KEY ((user_id, session_id), item_in_session))


In [55]:
try:
    session.execute(query)
except Exception as e:
    print(e)

In [56]:
query = "INSERT INTO music_library_user (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
query += "VALUES (%s, %s, %s, %s, %s, %s, %s)"
print(query)

INSERT INTO music_library_user (user_id, session_id, item_in_session, artist, song, first_name, last_name) VALUES (%s, %s, %s, %s, %s, %s, %s)


In [57]:
for line in read_event_data():
    try:
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
    except Exception as e:
        print(e)

#### Query 2 Description
This query returns artist, song, first_name, and last_name from the music_library_user table for user_id of 10 and session_id equal to 182. A composite partition key on the music_library_user table consists of user_id and session_id which enables fast read-access of data when specifying the user_id and session_id values. The rows within the partition are sorted by item_in_session in ascending order.

In [58]:
query = "SELECT artist, song, first_name, last_name FROM music_library_user WHERE user_id=10 and session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


---
### Query 3
#### Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [59]:
query = "CREATE TABLE IF NOT EXISTS music_library_song (song varchar, user_id int, first_name varchar, \
last_name varchar, PRIMARY KEY (song, last_name, first_name, user_id))"
print(query)

CREATE TABLE IF NOT EXISTS music_library_song (song varchar, user_id int, first_name varchar, last_name varchar, PRIMARY KEY (song, last_name, first_name, user_id))


In [60]:
try:
    session.execute(query)
except Exception as e:
    print(e)

In [61]:
query = "INSERT INTO music_library_song (song, user_id, first_name, last_name) "
query += "VALUES (%s, %s, %s, %s)"
print(query)

INSERT INTO music_library_song (song, user_id, first_name, last_name) VALUES (%s, %s, %s, %s)


In [62]:
for line in read_event_data():
    try:
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
    except Exception as e:
        print(e)

#### Query 3 Description
This query returns all users who listened to a particular song. The song is the partition key. Multiple listeners exist in each song partition across all nodes in the Cassandra cluster. The rows within each node partition are sorted by last name, then by first name and user_id in ascending order.

In [63]:
query="SELECT first_name, last_name FROM music_library_song WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Sara Johnson
Tegan Levine
Jacqueline Lynch


# Part IV. Shutdown

In [64]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [65]:
query = "drop table music_library_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [66]:
query = "drop table music_library_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [67]:
session.shutdown()
cluster.shutdown()